<a href="https://colab.research.google.com/github/mordor-ai/M2-fouille-donn-es/blob/master/Spark_Basic_Queries.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importing Spark

In [1]:
!pip install pyspark

In [2]:

import pyspark

from pyspark import SparkContext
from pyspark.sql import SparkSession

sc = SparkContext.getOrCreate()
spark = SparkSession.builder.appName("Python Spark").getOrCreate()

# Loading the data

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········


In [7]:
df_ratings = spark.read\
    .option("delimiter", "\t")\
    .option("header", "true")\
    .option("inferSchema", "true")\
    .csv('/content/drive/My Drive/data/u.data')

In [8]:
# print the dataframe schema
df_ratings.printSchema()

root
 |-- user_id: integer (nullable = true)
 |-- item_id: integer (nullable = true)
 |-- rating: integer (nullable = true)
 |-- timestamp: integer (nullable = true)



In [9]:
# show a sample of the data (the dataframe executes the whole pipeline at this stage)
df_ratings.show(5)

+-------+-------+------+---------+
|user_id|item_id|rating|timestamp|
+-------+-------+------+---------+
|    196|    242|     3|881250949|
|    186|    302|     3|891717742|
|     22|    377|     1|878887116|
|    244|     51|     2|880606923|
|    166|    346|     1|886397596|
+-------+-------+------+---------+
only showing top 5 rows



In [14]:
# set the rdd equivalent of the dataframe
rdd_ratings = df_ratings.rdd
rdd_ratings.take(15)

[Row(user_id=196, item_id=242, rating=3, timestamp=881250949),
 Row(user_id=186, item_id=302, rating=3, timestamp=891717742),
 Row(user_id=22, item_id=377, rating=1, timestamp=878887116),
 Row(user_id=244, item_id=51, rating=2, timestamp=880606923),
 Row(user_id=166, item_id=346, rating=1, timestamp=886397596),
 Row(user_id=298, item_id=474, rating=4, timestamp=884182806),
 Row(user_id=115, item_id=265, rating=2, timestamp=881171488),
 Row(user_id=253, item_id=465, rating=5, timestamp=891628467),
 Row(user_id=305, item_id=451, rating=3, timestamp=886324817),
 Row(user_id=6, item_id=86, rating=3, timestamp=883603013),
 Row(user_id=62, item_id=257, rating=2, timestamp=879372434),
 Row(user_id=286, item_id=1014, rating=5, timestamp=879781125),
 Row(user_id=200, item_id=222, rating=5, timestamp=876042340),
 Row(user_id=210, item_id=40, rating=3, timestamp=891035994),
 Row(user_id=224, item_id=29, rating=3, timestamp=888104457)]

# Basic Queries

#### Exercice 1 - Number of movies per user (using RDD then Dataframe)

Calculer pour chaque utilisateur le nombre de films notés, et afficher le résultat pour l'un d'entre eux. Utilisez dans un premier temps les RDD puis les Dataframes.

In [ ]:
# classical RDD approach

def map_function(row)
  print row(row.user_id)
  return row.user_id ,1
# on récupère le calcul count pour le premier user
# reduceByKey :  recupère deux tuples ayant les mêmes clefs et les additionne
u = rdd_ratings.map(map_function).reduceByKey(lambda v1, v2 :v1+v2).take(1)

In [ ]:
# dataframe approach, filtering on the previous user to compare results
# u[0][0] est le premier utilisateur
df_ratings.filter(df_ratings.user_id== u[0][0]).groupBy('user_id').count().show()

#### Exercice 2 - Average rating per user (using RDD then Dataframe)

Calculer pour chaque utilisateurs la note moyenne donnée et afficher le résultat pour l'un d'entre eux. Utilisez dans un premier temps les RDD puis les Dataframes.

In [ ]:
# classical RDD approach
rdd_temp = rdd_ratings.map(lambda : r: (r[0],r[2])).aggregateByKey((0,0), lambda a,b: (a[0]+b,a[1]+1),
                                                        lambda a,b: (a[0]+b[0],a[1]+b[1]))
rdd_temp.mapValues(lambda v: v[0]/v[1] ).take(5)

In [ ]:
# dataframe approach
df_ratings.groupBy('user_id').mean('rating').show()

#### Exercice 3 - Top-5 movies with at least 15 votes (Dataframe)

Afficher les 5 meilleurs films parmi ceux qui ont reçu au moins 15 votes.

*Indices:*
* Utiliser df_ratings pour calculer la moyenne, filtrer les films qui ont moins de 15 notes et classer les films par ordre décroissant.
* Faire un join avec df_movies pour afficher le nom des films sélectionnés.

In [ ]:
df_items = spark.read\
    .option("delimiter", "|")\
    .option("header", "true")\
    .option("inferSchema", "true")\
    .csv('data/u.item')